In [1]:
import importlib
import os
from pathlib import Path
import sys

from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
import pandas as pd

from dm import Country, DemographicModeling
from dm.utils import get_countries, add_enrich_aliases

# import arcpy if available
if importlib.util.find_spec("arcpy") is not None:
    import arcpy

In [2]:
# paths to common data locations - NOTE: to convert any path to a raw string, simply use str(path_instance)
dir_prj = Path.cwd().parent

dir_data = dir_prj/'data'
dir_raw = dir_data/'raw'
dir_int = dir_data/'interim'

gdb_raw = dir_raw/'raw.gdb'
gdb_int = dir_int/'interim.gdb'

# import the project package from the project package path - only necessary if you are not using a unique environemnt for this project
sys.path.append(str(dir_prj/'src'))
import canada_modeling

# load the "autoreload" extension so that code can change, & always reload modules so that as you change code in src, it gets loaded
%load_ext autoreload
%autoreload 2

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [3]:
get_countries('local')

,geo_ref,country,year
0,CAN_ESRI_2019,CAN,2019
1,USA_ESRI_2019,USA,2019
2,USA_ESRI_2020,USA,2020


In [4]:
can = Country('CAN')

can

<dm.Country - CAN (local)>

In [5]:
can.geographies

,geo_name,geo_alias,col_id,col_name,feature_class_path
0,disseminationareas,DisseminationAreas,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
1,censustracts,CensusTracts,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
2,fsas,FSAs,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
3,censussubdivisions,CensusSubdivisions,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
4,feds,FEDs,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
5,cmacas,CMACAs,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
6,censusdivisions,CensusDivisions,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
7,provinceterritories,ProvinceTerritories,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...
8,country,Country,ID,NAME,D:\arcgis\ba_data\can_2019\Data\Demographic Da...


In [6]:
van_df = can.cmacas.get('vancouver')

van_df

,ID,NAME,SHAPE
0,933,Vancouver,"{""rings"": [[[-122.89240540549281, 49.087271790..."


In [7]:
van_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

In [8]:
lvl0_df = van_df.dm.level(0).get()

lvl0_df.head()

,ID,NAME,SHAPE
0,59150369,59150369,"{""rings"": [[[-123.0705300003746, 49.2525500003..."
1,59150370,59150370,"{""rings"": [[[-123.07615000003094, 49.251719999..."
2,59150373,59150373,"{""rings"": [[[-123.08187000014466, 49.250950000..."
3,59150375,59150375,"{""rings"": [[[-123.08198000064783, 49.247739999..."
4,59150376,59150376,"{""rings"": [[[-123.08268000025659, 49.246560000..."


In [9]:
lvl0_df.spatial.plot()

MapView(layout=Layout(height='400px', width='100%'))

In [10]:
evars = can.enrich_variables

keyvars = evars[(evars.data_collection.str.startswith('Key')) & (evars.vintage == '2022')].copy().reset_index(drop=True)

keyvars

,name,alias,type,vintage,data_collection,enrich_str,enrich_field_name
0,P3YPTAPOP,2022 Total Population,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YPTAPOP,KeyCanFacts_P3YPTAPOP
1,P3YHFSCF,2022 Tot Census Fam HHs,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YHFSCF,KeyCanFacts_P3YHFSCF
2,P3YHSZAVG,2022 Private HHs Avg Num Persons,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YHSZAVG,KeyCanFacts_P3YHSZAVG
3,P3YHNIAVG,2022 HH Inc Average Curr$,CURRENCY,2022,KeyCanFacts,KeyCanFacts.P3YHNIAVG,KeyCanFacts_P3YHNIAVG
4,P3YHNIMED,2022 HH Inc Median Curr$,CURRENCY,2022,KeyCanFacts,KeyCanFacts.P3YHNIMED,KeyCanFacts_P3YHNIMED
5,P3YTENHHD,2022 Tenure - Total HHs,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENHHD,KeyCanFacts_P3YTENHHD
6,P3YTENOWN,2022 Tenure - Owned,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENOWN,KeyCanFacts_P3YTENOWN
7,P3YTENRENT,2022 Tenure - Rented,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENRENT,KeyCanFacts_P3YTENRENT
8,P3YTENBAND,2022 Tenure - Band Housing,COUNT,2022,KeyCanFacts,KeyCanFacts.P3YTENBAND,KeyCanFacts_P3YTENBAND


In [11]:
enrich_df = lvl0_df.dm.enrich(keyvars)

enrich_df.head()

,ID,NAME,KeyCanFacts_P3YPTAPOP,KeyCanFacts_P3YHFSCF,KeyCanFacts_P3YHSZAVG,KeyCanFacts_P3YHNIAVG,KeyCanFacts_P3YHNIMED,KeyCanFacts_P3YTENHHD,KeyCanFacts_P3YTENOWN,KeyCanFacts_P3YTENRENT,KeyCanFacts_P3YTENBAND,SHAPE
0,59150369,59150369,596.0,148.0,2.6,90408.28,71666.67,225.0,116.0,109.0,0.0,"{""rings"": [[[-123.0705300003746, 49.2525500003..."
1,59150370,59150370,1498.0,393.0,2.1,93374.01,82450.98,705.0,357.0,348.0,0.0,"{""rings"": [[[-123.07615000003094, 49.251719999..."
2,59150373,59150373,767.0,201.0,3.0,148982.53,117672.41,253.0,199.0,54.0,0.0,"{""rings"": [[[-123.08187000014466, 49.250950000..."
3,59150375,59150375,507.0,130.0,3.0,141579.89,102343.75,167.0,115.0,52.0,0.0,"{""rings"": [[[-123.08198000064783, 49.247739999..."
4,59150376,59150376,497.0,128.0,2.9,130564.61,101973.68,173.0,100.0,73.0,0.0,"{""rings"": [[[-123.08268000025659, 49.246560000..."


In [12]:
enrich_fc = enrich_df.spatial.to_featureclass(gdb_int/'van_lvl0_keyvars')

enrich_fc

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\van_lvl0_keyvars'

In [13]:
add_enrich_aliases(enrich_fc, can)

'D:\\projects\\canada-modeling\\data\\interim\\interim.gdb\\van_lvl0_keyvars'